In [1]:
!pip show webdriver_manager

Name: webdriver-manager
Version: 3.8.5
Summary: Library provides the way to automatically manage drivers for different browsers
Home-page: https://github.com/SergeyPirogov/webdriver_manager
Author: Sergey Pirogov
Author-email: automationremarks@gmail.com
License: 
Location: c:\users\admin\anaconda3\lib\site-packages
Requires: packaging, tqdm, python-dotenv, requests
Required-by: 


In [2]:
import pandas as pd
import numpy as np
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import lxml
import json
import time
import csv

In [3]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [4]:
service = Service(executable_path=ChromeDriverManager().install())

In [6]:
#driver = webdriver.Chrome(service=service)

In [32]:
# driver.get(lnk)
# html = driver.page_source

In [5]:
# Формируем суп-объект

def get_soup(url):    
    driver.get(url)    
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    return soup

In [6]:
# Извлекаем из soup-объекта название и ссылку с id фильма

def links_parse(row):
    page = []
    time.sleep(5)
    try:
        soup = get_soup(row)
        block_pos = soup.select("div.styles_upper__j8BIs a")
        for i in block_pos:    
            name_pos = i.select_one("div.base-movie-main-info_mainInfo__ZL_u3").get_text()
            link_pos = i.get('href')
            dic = {'name': name_pos, 'link': link_pos}  
            page.append(dic)
    except:
        page = []
    return page

In [7]:
# ЗАПУСК ПАРСИНГА
# - год меняется в теле ссылки после year--
# - количество страниц пагинации меняется в параметрах цикла range внутри скобок

link = 'https://www.kinopoisk.ru/lists/movies/year--2020/?page={}'
year = []
driver = webdriver.Chrome(service=service) # Открывает новую отдельную вкладку браузера хром от selenium
for i in range(1, 1193):
    ln = link.format(i)
    inf = links_parse(ln)
    year.extend(inf)
driver.quit()   # Закрывает отдельную вкладку браузера хром от selenium после выполнения цикла

In [8]:
# Записываем результаты в датафрейм

df_2020 = pd.DataFrame(year)

In [10]:
# Прибавляем хвост ссылки к домену

df_2020['links'] = df_2020['link'].apply(lambda x: 'https://www.kinopoisk.ru' + x)
df_2020 = df_2020[['name', 'links']]

In [11]:
df_2020

,name,links
0,Рик и Морти,https://www.kinopoisk.ru/series/685246/
1,Триггер,https://www.kinopoisk.ru/series/1100777/
2,Беспринципные,https://www.kinopoisk.ru/series/1355059/
3,Душа,https://www.kinopoisk.ru/film/775273/
4,Маша и Медведь,https://www.kinopoisk.ru/series/478491/
...,...,...
51644,Never Ever,https://www.kinopoisk.ru/series/1366143/
51645,Партия,https://www.kinopoisk.ru/series/957853/
51646,Крым,https://www.kinopoisk.ru/series/982486/
51647,СССР,https://www.kinopoisk.ru/series/1040584/


In [13]:
df_2020.to_csv('D:\\Data\\Kinopoisk\\csv\\df_2020.csv')

In [8]:
driver.quit()